In [6]:
import sys
import pandas as pd
import numpy as np
from pathlib import Path

project_root = Path().resolve().parent  

if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
    sys.path.append(str("/Users/seydaaybar/Desktop/ntt_data/python_files"))

from python_files.config import CFG
from python_files.helpers import load_selected_data, merge_transaction_data,check_missing_and_infinite, plot_distribution, analyze_numeric_column_over_time, drop_negative_values

In [7]:
data = load_selected_data(CFG, ['df_test'])

df_test = data['df_test']
df_test


,shop,item
0,101,100031
1,101,100486
2,101,100787
3,101,100794
4,101,100968
...,...,...
31526,158,122087
31527,158,122088
31528,158,122091
31529,158,122100


In [8]:
test_df = df_test.rename(columns={'shop': 'shop_id', 'item': 'item_id'})
test_df

,shop_id,item_id
0,101,100031
1,101,100486
2,101,100787
3,101,100794
4,101,100968
...,...,...
31526,158,122087
31527,158,122088
31528,158,122091
31529,158,122100


In [9]:
df_new = pd.read_csv('monthly_sales_cb.csv')
df_new

,shop_id,item_id,date,sales
0,99,100030,2013-02-28,26691.0
1,99,100031,2013-02-28,15510.0
2,99,100032,2013-01-31,4308.0
3,99,100032,2013-02-28,7180.0
4,99,100033,2013-01-31,3381.0
...,...,...,...,...
1576700,158,122164,2015-04-30,4540.0
1576701,158,122164,2015-07-31,2270.0
1576702,158,122167,2013-10-31,972.0
1576703,158,122167,2013-12-31,1944.0


In [10]:
df_new

,shop_id,item_id,date,sales
0,99,100030,2013-02-28,26691.0
1,99,100031,2013-02-28,15510.0
2,99,100032,2013-01-31,4308.0
3,99,100032,2013-02-28,7180.0
4,99,100033,2013-01-31,3381.0
...,...,...,...,...
1576700,158,122164,2015-04-30,4540.0
1576701,158,122164,2015-07-31,2270.0
1576702,158,122167,2013-10-31,972.0
1576703,158,122167,2013-12-31,1944.0


In [12]:
df_new = pd.read_csv('monthly_sales_cb.csv')
df_new['date'] = pd.to_datetime(df_new['date'])
df_new = df_new.sort_values(['shop_id', 'item_id', 'date']).reset_index(drop=True)

lag_periods = [1, 2, 3, 6, 12]
for lag in lag_periods:
    df_new[f'lag_{lag}'] = df_new.groupby(['shop_id', 'item_id'])['sales'].shift(lag)

lag_cols = [f'lag_{lag}' for lag in lag_periods]
df_new[lag_cols] = df_new[lag_cols].fillna(0)

df_new['rolling_mean_3'] = (
    df_new.groupby(['shop_id', 'item_id'])['sales']
    .transform(lambda x: x.shift(1).rolling(window=3, min_periods=1).mean())
).fillna(0)

df_new['trend_1_2'] = df_new['lag_1'] - df_new['lag_2']
df_new['lag_1_ratio_2'] = df_new['lag_1'] / (df_new['lag_2'] + 1)

df_new['month'] = df_new['date'].dt.month
df_new['year'] = df_new['date'].dt.year
df_new['quarter'] = df_new['date'].dt.quarter
df_new['is_month_start'] = df_new['date'].dt.is_month_start.astype(int)
df_new['is_month_end'] = df_new['date'].dt.is_month_end.astype(int)

def get_season(month):
    if month in [12, 1, 2]:
        return 'winter'
    elif month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    else:
        return 'fall'

df_new['season'] = df_new['month'].apply(get_season)

In [13]:
df_new.to_csv('monthly_sales_with_features.csv', index=False)

Find the last known date per (shop_id, item_id) pair in  historical data

In [8]:
last_dates = df_new.groupby(['shop_id', 'item_id'])['date'].max().reset_index()
last_dates = last_dates.rename(columns={'date': 'last_date'})
last_dates

,shop_id,item_id,last_date
0,99,100030,2013-02-28
1,99,100031,2013-02-28
2,99,100032,2013-02-28
3,99,100033,2013-02-28
4,99,100035,2013-02-28
...,...,...,...
416973,158,122154,2013-01-31
416974,158,122155,2013-07-31
416975,158,122162,2015-08-31
416976,158,122164,2015-07-31


 Merge these last dates with  test pairs

In [9]:
test_pairs_with_dates = pd.merge(test_df, last_dates, on=['shop_id', 'item_id'], how='left')
test_pairs_with_dates

,shop_id,item_id,last_date
0,101,100031,2014-05-31
1,101,100486,2015-09-30
2,101,100787,2015-03-31
3,101,100794,NaT
4,101,100968,NaT
...,...,...,...
31526,158,122087,2015-09-30
31527,158,122088,2015-09-30
31528,158,122091,2015-09-30
31529,158,122100,2015-09-30


Fill missing last dates for pairs without historical data

In [10]:
global_max_date = df_new['date'].max()
test_pairs_with_dates['last_date'] = test_pairs_with_dates['last_date'].fillna(global_max_date)
test_pairs_with_dates

,shop_id,item_id,last_date
0,101,100031,2014-05-31
1,101,100486,2015-09-30
2,101,100787,2015-03-31
3,101,100794,2015-09-30
4,101,100968,2015-09-30
...,...,...,...
31526,158,122087,2015-09-30
31527,158,122088,2015-09-30
31528,158,122091,2015-09-30
31529,158,122100,2015-09-30


In [37]:
test_pairs_with_dates.isna().sum()

shop_id      0
item_id      0
last_date    0
dtype: int64

In [11]:
df_new['date'] = pd.to_datetime(df_new['date'])
test_pairs_with_dates['last_date'] = pd.to_datetime(test_pairs_with_dates['last_date'])


In [12]:
test_pairs_with_dates['last_date'] = test_pairs_with_dates['last_date'].fillna(global_max_date)


In [13]:
test_pairs_with_dates

,shop_id,item_id,last_date
0,101,100031,2014-05-31
1,101,100486,2015-09-30
2,101,100787,2015-03-31
3,101,100794,2015-09-30
4,101,100968,2015-09-30
...,...,...,...
31526,158,122087,2015-09-30
31527,158,122088,2015-09-30
31528,158,122091,2015-09-30
31529,158,122100,2015-09-30


 Get the full feature row for each pair at their last known date

In [49]:
last_features = pd.merge(
    df_new,
    test_pairs_with_dates,
    left_on=['shop_id', 'item_id', 'date'],
    right_on=['shop_id', 'item_id', 'last_date'],
    how='right'
)



In [51]:
last_features.isna().sum()

shop_id              0
item_id              0
date              7120
sales             7120
lag_1             7120
lag_2             7120
lag_3             7120
lag_6             7120
lag_12            7120
rolling_mean_3    7120
trend_1_2         7120
lag_1_ratio_2     7120
month             7120
year              7120
quarter           7120
is_month_start    7120
is_month_end      7120
season            7120
last_date            0
dtype: int64

In [52]:
last_features

,shop_id,item_id,date,sales,lag_1,lag_2,lag_3,lag_6,lag_12,rolling_mean_3,trend_1_2,lag_1_ratio_2,month,year,quarter,is_month_start,is_month_end,season,last_date
0,101,100031,2014-05-31,1351.0,2270.0,2269.0,9080.0,0.0,0.0,4539.666667,1.0,1.000000,5.0,2014.0,2.0,0.0,1.0,spring,2014-05-31
1,101,100486,2015-09-30,975.0,1950.0,975.0,1950.0,975.0,0.0,1625.000000,975.0,1.997951,9.0,2015.0,3.0,0.0,1.0,fall,2015-09-30
2,101,100787,2015-03-31,1364.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,3.0,2015.0,1.0,0.0,1.0,spring,2015-03-31
3,101,100794,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-30
4,101,100968,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31526,158,122087,2015-09-30,1161.0,774.0,1935.0,387.0,1935.0,1028.0,1032.000000,-1161.0,0.399793,9.0,2015.0,3.0,0.0,1.0,fall,2015-09-30
31527,158,122088,2015-09-30,387.0,2709.0,2709.0,1548.0,1548.0,2056.0,2322.000000,0.0,0.999631,9.0,2015.0,3.0,0.0,1.0,fall,2015-09-30
31528,158,122091,2015-09-30,1746.0,517.0,452.0,4520.0,1065.0,710.0,1829.666667,65.0,1.141280,9.0,2015.0,3.0,0.0,1.0,fall,2015-09-30
31529,158,122100,2015-09-30,2043.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,9.0,2015.0,3.0,0.0,1.0,fall,2015-09-30


In [53]:
last_features[lag_cols] = last_features[lag_cols].fillna(0)
last_features['rolling_mean_3'] = last_features['rolling_mean_3'].fillna(0)
last_features['trend_1_2'] = last_features['trend_1_2'].fillna(0)
last_features['lag_1_ratio_2'] = last_features['lag_1_ratio_2'].fillna(0)
last_features

,shop_id,item_id,date,sales,lag_1,lag_2,lag_3,lag_6,lag_12,rolling_mean_3,trend_1_2,lag_1_ratio_2,month,year,quarter,is_month_start,is_month_end,season,last_date
0,101,100031,2014-05-31,1351.0,2270.0,2269.0,9080.0,0.0,0.0,4539.666667,1.0,1.000000,5.0,2014.0,2.0,0.0,1.0,spring,2014-05-31
1,101,100486,2015-09-30,975.0,1950.0,975.0,1950.0,975.0,0.0,1625.000000,975.0,1.997951,9.0,2015.0,3.0,0.0,1.0,fall,2015-09-30
2,101,100787,2015-03-31,1364.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,3.0,2015.0,1.0,0.0,1.0,spring,2015-03-31
3,101,100794,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-30
4,101,100968,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31526,158,122087,2015-09-30,1161.0,774.0,1935.0,387.0,1935.0,1028.0,1032.000000,-1161.0,0.399793,9.0,2015.0,3.0,0.0,1.0,fall,2015-09-30
31527,158,122088,2015-09-30,387.0,2709.0,2709.0,1548.0,1548.0,2056.0,2322.000000,0.0,0.999631,9.0,2015.0,3.0,0.0,1.0,fall,2015-09-30
31528,158,122091,2015-09-30,1746.0,517.0,452.0,4520.0,1065.0,710.0,1829.666667,65.0,1.141280,9.0,2015.0,3.0,0.0,1.0,fall,2015-09-30
31529,158,122100,2015-09-30,2043.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,9.0,2015.0,3.0,0.0,1.0,fall,2015-09-30


In [54]:
test_next_month = last_features.copy()
test_next_month['date'] = test_next_month['date'] + pd.DateOffset(months=1)
test_next_month

,shop_id,item_id,date,sales,lag_1,lag_2,lag_3,lag_6,lag_12,rolling_mean_3,trend_1_2,lag_1_ratio_2,month,year,quarter,is_month_start,is_month_end,season,last_date
0,101,100031,2014-06-30,1351.0,2270.0,2269.0,9080.0,0.0,0.0,4539.666667,1.0,1.000000,5.0,2014.0,2.0,0.0,1.0,spring,2014-05-31
1,101,100486,2015-10-30,975.0,1950.0,975.0,1950.0,975.0,0.0,1625.000000,975.0,1.997951,9.0,2015.0,3.0,0.0,1.0,fall,2015-09-30
2,101,100787,2015-04-30,1364.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,3.0,2015.0,1.0,0.0,1.0,spring,2015-03-31
3,101,100794,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-30
4,101,100968,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31526,158,122087,2015-10-30,1161.0,774.0,1935.0,387.0,1935.0,1028.0,1032.000000,-1161.0,0.399793,9.0,2015.0,3.0,0.0,1.0,fall,2015-09-30
31527,158,122088,2015-10-30,387.0,2709.0,2709.0,1548.0,1548.0,2056.0,2322.000000,0.0,0.999631,9.0,2015.0,3.0,0.0,1.0,fall,2015-09-30
31528,158,122091,2015-10-30,1746.0,517.0,452.0,4520.0,1065.0,710.0,1829.666667,65.0,1.141280,9.0,2015.0,3.0,0.0,1.0,fall,2015-09-30
31529,158,122100,2015-10-30,2043.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,9.0,2015.0,3.0,0.0,1.0,fall,2015-09-30


In [58]:
test_next_month["date"] = test_next_month["last_date"]
test_next_month

,shop_id,item_id,date,lag_1,lag_2,lag_3,lag_6,lag_12,rolling_mean_3,trend_1_2,lag_1_ratio_2,month,year,quarter,is_month_start,is_month_end,season,last_date
0,101,100031,2014-05-31,2270.0,2269.0,9080.0,0.0,0.0,4539.666667,1.0,1.000000,6.0,2014.0,2.0,0,1,summer,2014-05-31
1,101,100486,2015-09-30,1950.0,975.0,1950.0,975.0,0.0,1625.000000,975.0,1.997951,10.0,2015.0,4.0,0,0,fall,2015-09-30
2,101,100787,2015-03-31,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,4.0,2015.0,2.0,0,1,spring,2015-03-31
3,101,100794,2015-09-30,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,NaN,NaN,NaN,0,0,fall,2015-09-30
4,101,100968,2015-09-30,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,NaN,NaN,NaN,0,0,fall,2015-09-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31526,158,122087,2015-09-30,774.0,1935.0,387.0,1935.0,1028.0,1032.000000,-1161.0,0.399793,10.0,2015.0,4.0,0,0,fall,2015-09-30
31527,158,122088,2015-09-30,2709.0,2709.0,1548.0,1548.0,2056.0,2322.000000,0.0,0.999631,10.0,2015.0,4.0,0,0,fall,2015-09-30
31528,158,122091,2015-09-30,517.0,452.0,4520.0,1065.0,710.0,1829.666667,65.0,1.141280,10.0,2015.0,4.0,0,0,fall,2015-09-30
31529,158,122100,2015-09-30,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,10.0,2015.0,4.0,0,0,fall,2015-09-30


In [60]:
test_next_month['month'] = test_next_month['date'].dt.month
test_next_month['year'] = test_next_month['date'].dt.year
test_next_month['quarter'] = test_next_month['date'].dt.quarter
test_next_month['is_month_start'] = test_next_month['date'].dt.is_month_start.astype(int)
test_next_month['is_month_end'] = test_next_month['date'].dt.is_month_end.astype(int)
test_next_month['season'] = test_next_month['month'].apply(get_season)
test_next_month

,shop_id,item_id,date,lag_1,lag_2,lag_3,lag_6,lag_12,rolling_mean_3,trend_1_2,lag_1_ratio_2,month,year,quarter,is_month_start,is_month_end,season,last_date
0,101,100031,2014-05-31,2270.0,2269.0,9080.0,0.0,0.0,4539.666667,1.0,1.000000,5,2014,2,0,1,spring,2014-05-31
1,101,100486,2015-09-30,1950.0,975.0,1950.0,975.0,0.0,1625.000000,975.0,1.997951,9,2015,3,0,1,fall,2015-09-30
2,101,100787,2015-03-31,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,3,2015,1,0,1,spring,2015-03-31
3,101,100794,2015-09-30,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,9,2015,3,0,1,fall,2015-09-30
4,101,100968,2015-09-30,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,9,2015,3,0,1,fall,2015-09-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31526,158,122087,2015-09-30,774.0,1935.0,387.0,1935.0,1028.0,1032.000000,-1161.0,0.399793,9,2015,3,0,1,fall,2015-09-30
31527,158,122088,2015-09-30,2709.0,2709.0,1548.0,1548.0,2056.0,2322.000000,0.0,0.999631,9,2015,3,0,1,fall,2015-09-30
31528,158,122091,2015-09-30,517.0,452.0,4520.0,1065.0,710.0,1829.666667,65.0,1.141280,9,2015,3,0,1,fall,2015-09-30
31529,158,122100,2015-09-30,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,9,2015,3,0,1,fall,2015-09-30


In [61]:
if 'sales' in test_next_month.columns:
    test_next_month = test_next_month.drop(columns=['sales'])
test_next_month

,shop_id,item_id,date,lag_1,lag_2,lag_3,lag_6,lag_12,rolling_mean_3,trend_1_2,lag_1_ratio_2,month,year,quarter,is_month_start,is_month_end,season,last_date
0,101,100031,2014-05-31,2270.0,2269.0,9080.0,0.0,0.0,4539.666667,1.0,1.000000,5,2014,2,0,1,spring,2014-05-31
1,101,100486,2015-09-30,1950.0,975.0,1950.0,975.0,0.0,1625.000000,975.0,1.997951,9,2015,3,0,1,fall,2015-09-30
2,101,100787,2015-03-31,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,3,2015,1,0,1,spring,2015-03-31
3,101,100794,2015-09-30,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,9,2015,3,0,1,fall,2015-09-30
4,101,100968,2015-09-30,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,9,2015,3,0,1,fall,2015-09-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31526,158,122087,2015-09-30,774.0,1935.0,387.0,1935.0,1028.0,1032.000000,-1161.0,0.399793,9,2015,3,0,1,fall,2015-09-30
31527,158,122088,2015-09-30,2709.0,2709.0,1548.0,1548.0,2056.0,2322.000000,0.0,0.999631,9,2015,3,0,1,fall,2015-09-30
31528,158,122091,2015-09-30,517.0,452.0,4520.0,1065.0,710.0,1829.666667,65.0,1.141280,9,2015,3,0,1,fall,2015-09-30
31529,158,122100,2015-09-30,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,9,2015,3,0,1,fall,2015-09-30


In [62]:
test_next_month = test_next_month.reset_index(drop=True)
test_next_month

,shop_id,item_id,date,lag_1,lag_2,lag_3,lag_6,lag_12,rolling_mean_3,trend_1_2,lag_1_ratio_2,month,year,quarter,is_month_start,is_month_end,season,last_date
0,101,100031,2014-05-31,2270.0,2269.0,9080.0,0.0,0.0,4539.666667,1.0,1.000000,5,2014,2,0,1,spring,2014-05-31
1,101,100486,2015-09-30,1950.0,975.0,1950.0,975.0,0.0,1625.000000,975.0,1.997951,9,2015,3,0,1,fall,2015-09-30
2,101,100787,2015-03-31,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,3,2015,1,0,1,spring,2015-03-31
3,101,100794,2015-09-30,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,9,2015,3,0,1,fall,2015-09-30
4,101,100968,2015-09-30,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,9,2015,3,0,1,fall,2015-09-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31526,158,122087,2015-09-30,774.0,1935.0,387.0,1935.0,1028.0,1032.000000,-1161.0,0.399793,9,2015,3,0,1,fall,2015-09-30
31527,158,122088,2015-09-30,2709.0,2709.0,1548.0,1548.0,2056.0,2322.000000,0.0,0.999631,9,2015,3,0,1,fall,2015-09-30
31528,158,122091,2015-09-30,517.0,452.0,4520.0,1065.0,710.0,1829.666667,65.0,1.141280,9,2015,3,0,1,fall,2015-09-30
31529,158,122100,2015-09-30,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,9,2015,3,0,1,fall,2015-09-30


In [65]:
test_next_month.isna().sum()


shop_id           0
item_id           0
date              0
lag_1             0
lag_2             0
lag_3             0
lag_6             0
lag_12            0
rolling_mean_3    0
trend_1_2         0
lag_1_ratio_2     0
month             0
year              0
quarter           0
is_month_start    0
is_month_end      0
season            0
last_date         0
dtype: int64

In [1]:
from preprocess import load_and_prepare_test_data

test_df, df_new, test_pairs_with_dates, last_features, test_next_month = load_and_prepare_test_data()

In [2]:
test_df

,shop_id,item_id
0,101,100031
1,101,100486
2,101,100787
3,101,100794
4,101,100968
...,...,...
31526,158,122087
31527,158,122088
31528,158,122091
31529,158,122100


In [3]:
df_new

,shop_id,item_id,date,sales,lag_1,lag_2,lag_3,lag_6,lag_12,rolling_mean_3,trend_1_2,lag_1_ratio_2,month,year,quarter,is_month_start,is_month_end,season
0,99,100030,2013-02-28,26691.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,2,2013,1,0,1,winter
1,99,100031,2013-02-28,15510.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,2,2013,1,0,1,winter
2,99,100032,2013-01-31,4308.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1,2013,1,0,1,winter
3,99,100032,2013-02-28,7180.0,4308.0,0.0,0.0,0.0,0.0,4308.000000,4308.0,4308.000000,2,2013,1,0,1,winter
4,99,100033,2013-01-31,3381.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1,2013,1,0,1,winter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1576700,158,122164,2015-04-30,4540.0,2433.0,4866.0,0.0,0.0,0.0,3649.500000,-2433.0,0.499897,4,2015,2,0,1,spring
1576701,158,122164,2015-07-31,2270.0,4540.0,2433.0,4866.0,0.0,0.0,3946.333333,2107.0,1.865242,7,2015,3,0,1,summer
1576702,158,122167,2013-10-31,972.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,10,2013,4,0,1,fall
1576703,158,122167,2013-12-31,1944.0,972.0,0.0,0.0,0.0,0.0,972.000000,972.0,972.000000,12,2013,4,0,1,winter


In [4]:
test_pairs_with_dates

,shop_id,item_id,last_date
0,101,100031,2014-05-31
1,101,100486,2015-09-30
2,101,100787,2015-03-31
3,101,100794,2015-09-30
4,101,100968,2015-09-30
...,...,...,...
31526,158,122087,2015-09-30
31527,158,122088,2015-09-30
31528,158,122091,2015-09-30
31529,158,122100,2015-09-30


In [5]:
last_features

,shop_id,item_id,date,sales,lag_1,lag_2,lag_3,lag_6,lag_12,rolling_mean_3,trend_1_2,lag_1_ratio_2,month,year,quarter,is_month_start,is_month_end,season,last_date
0,101,100031,2014-05-31,1351.0,2270.0,2269.0,9080.0,0.0,0.0,4539.666667,1.0,1.000000,5.0,2014.0,2.0,0.0,1.0,spring,2014-05-31
1,101,100486,2015-09-30,975.0,1950.0,975.0,1950.0,975.0,0.0,1625.000000,975.0,1.997951,9.0,2015.0,3.0,0.0,1.0,fall,2015-09-30
2,101,100787,2015-03-31,1364.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,3.0,2015.0,1.0,0.0,1.0,spring,2015-03-31
3,101,100794,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-30
4,101,100968,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31526,158,122087,2015-09-30,1161.0,774.0,1935.0,387.0,1935.0,1028.0,1032.000000,-1161.0,0.399793,9.0,2015.0,3.0,0.0,1.0,fall,2015-09-30
31527,158,122088,2015-09-30,387.0,2709.0,2709.0,1548.0,1548.0,2056.0,2322.000000,0.0,0.999631,9.0,2015.0,3.0,0.0,1.0,fall,2015-09-30
31528,158,122091,2015-09-30,1746.0,517.0,452.0,4520.0,1065.0,710.0,1829.666667,65.0,1.141280,9.0,2015.0,3.0,0.0,1.0,fall,2015-09-30
31529,158,122100,2015-09-30,2043.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,9.0,2015.0,3.0,0.0,1.0,fall,2015-09-30


In [6]:
test_next_month

,shop_id,item_id,date,lag_1,lag_2,lag_3,lag_6,lag_12,rolling_mean_3,trend_1_2,lag_1_ratio_2,month,year,quarter,is_month_start,is_month_end,season
0,101,100031,2014-05-31,2270.0,2269.0,9080.0,0.0,0.0,4539.666667,1.0,1.000000,5,2014,2,0,1,spring
1,101,100486,2015-09-30,1950.0,975.0,1950.0,975.0,0.0,1625.000000,975.0,1.997951,9,2015,3,0,1,fall
2,101,100787,2015-03-31,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,3,2015,1,0,1,spring
3,101,100794,2015-09-30,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,9,2015,3,0,1,fall
4,101,100968,2015-09-30,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,9,2015,3,0,1,fall
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31526,158,122087,2015-09-30,774.0,1935.0,387.0,1935.0,1028.0,1032.000000,-1161.0,0.399793,9,2015,3,0,1,fall
31527,158,122088,2015-09-30,2709.0,2709.0,1548.0,1548.0,2056.0,2322.000000,0.0,0.999631,9,2015,3,0,1,fall
31528,158,122091,2015-09-30,517.0,452.0,4520.0,1065.0,710.0,1829.666667,65.0,1.141280,9,2015,3,0,1,fall
31529,158,122100,2015-09-30,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,9,2015,3,0,1,fall


In [1]:
df_new.unique().count()

NameError: name 'df_new' is not defined

## LSTM TEST DATA PREP

#### Check for unmatched (shop, item) pairs in test data

In [40]:
df_monthly_lstm = pd.read_csv("df_monthly_lstm.csv")
df_monthly_lstm_encfinal = pd.read_csv("df_monthly_lstm_encfinal.csv")
df_monthly_lstm_enc2 = pd.read_csv("df_monthly_lstm_enc2.csv")

In [72]:

train_pairs = set(zip(df_monthly_lstm['shop_id'], df_monthly_lstm['item_id']))

df_test['pair'] = list(zip(df_test['shop'], df_test['item']))

unmatched_pairs = df_test[~df_test['pair'].isin(train_pairs)]

print(f" Total test pairs: {len(df_test)}")
print(f" Unmatched pairs not seen in training: {len(unmatched_pairs)}")
print(f"Matched pairs: {len(df_test) - len(unmatched_pairs)}")


print("\nSample unmatched pairs:")
print(unmatched_pairs[['shop', 'item']].head())

df_test_filtered = df_test[df_test['pair'].isin(train_pairs)].reset_index(drop=True)

print(f"\nTest set after filtering: {df_test_filtered.shape}")


 Total test pairs: 31531
 Unmatched pairs not seen in training: 7120
Matched pairs: 24411

Sample unmatched pairs:
    shop    item
3    101  100794
4    101  100968
5    101  100988
7    101  101121
22   101  101578

Test set after filtering: (24411, 3)


In [74]:
df_test_filtered = df_test_filtered.drop(columns=["pair"], errors='ignore')
df_test_filtered

,shop,item
0,101,100031
1,101,100486
2,101,100787
3,101,101075
4,101,101377
...,...,...
24406,158,122087
24407,158,122088
24408,158,122091
24409,158,122100


In [75]:
# Load encoders
import joblib
le_shop = joblib.load('le_shop.pkl')
le_item = joblib.load('le_item.pkl')
le_cat = joblib.load('le_cat.pkl')  # You will need this later if category is used

# Encode test data
df_test_filtered['shop_id_enc'] = le_shop.transform(df_test_filtered['shop'])
df_test_filtered['item_id_enc'] = le_item.transform(df_test_filtered['item'])


In [76]:
df_test_filtered

,shop,item,shop_id_enc,item_id_enc
0,101,100031,2,31
1,101,100486,2,442
2,101,100787,2,713
3,101,101075,2,992
4,101,101377,2,1286
...,...,...,...,...
24406,158,122087,58,21248
24407,158,122088,58,21249
24408,158,122091,58,21252
24409,158,122100,58,21261


### Add Category Information

In [77]:
df_test_filtered.rename(columns={'item':'item_id', 'shop': 'shop_id'}, inplace=True)
df_test_filtered

,shop_id,item_id,shop_id_enc,item_id_enc
0,101,100031,2,31
1,101,100486,2,442
2,101,100787,2,713
3,101,101075,2,992
4,101,101377,2,1286
...,...,...,...,...
24406,158,122087,58,21248
24407,158,122088,58,21249
24408,158,122091,58,21252
24409,158,122100,58,21261


In [78]:
item_category_lookup = df_monthly_lstm[['item_id', 'item_category_id']].drop_duplicates()

df_test_filtered = df_test_filtered.merge(item_category_lookup, on='item_id', how='left')

df_test_filtered['item_category_enc'] = le_cat.transform(df_test_filtered['item_category_id'])
df_test_filtered

,shop_id,item_id,shop_id_enc,item_id_enc,item_category_id,item_category_enc
0,101,100031,2,31,1037,37
1,101,100486,2,442,1073,73
2,101,100787,2,713,1049,49
3,101,101075,2,992,1040,40
4,101,101377,2,1286,1023,23
...,...,...,...,...,...,...
24406,158,122087,58,21248,1083,83
24407,158,122088,58,21249,1083,83
24408,158,122091,58,21252,1083,83
24409,158,122100,58,21261,1042,42


In [38]:
feature_cols = ['amount', 'price_log', 'sales_log']
window_size = 12  

In [79]:
df_monthly_lstm_encfinal = df_monthly_lstm_encfinal.drop(columns=["date"], errors="ignore" )
df_monthly_lstm_encfinal

,amount,shop_id_enc,item_id_enc,item_category_enc,sales_log,price_log
0,3.492692,0,30,40,2.014386,-0.438664
1,1.058492,0,31,37,1.592230,0.055798
2,0.449942,0,32,40,0.596098,-0.620670
3,0.936782,0,32,40,0.993303,-0.620670
4,0.084812,0,33,37,0.407705,-0.168810
...,...,...,...,...,...,...
1576700,-0.036898,58,21323,37,0.636882,0.533327
1576701,-0.158608,58,21323,37,0.097982,0.533327
1576702,-0.158608,58,21326,49,-0.561204,-0.317126
1576703,-0.036898,58,21326,49,-0.022532,-0.317126


In [80]:
df_test_filtered

,shop_id,item_id,shop_id_enc,item_id_enc,item_category_id,item_category_enc
0,101,100031,2,31,1037,37
1,101,100486,2,442,1073,73
2,101,100787,2,713,1049,49
3,101,101075,2,992,1040,40
4,101,101377,2,1286,1023,23
...,...,...,...,...,...,...
24406,158,122087,58,21248,1083,83
24407,158,122088,58,21249,1083,83
24408,158,122091,58,21252,1083,83
24409,158,122100,58,21261,1042,42


In [82]:
df_test_filtered.drop(columns=["item_category_enc_x", "item_category_enc_y", "shop_id", "item_id", "item_category_id"], errors="ignore")

,shop_id_enc,item_id_enc,item_category_enc
0,2,31,37
1,2,442,73
2,2,713,49
3,2,992,40
4,2,1286,23
...,...,...,...
24406,58,21248,83
24407,58,21249,83
24408,58,21252,83
24409,58,21261,42


In [83]:
import numpy as np

def get_test_sequences(test_df, train_df, feature_cols, window_size):
    sequences = []
    valid_indices = []

    for idx, row in test_df.iterrows():
        shop = row['shop_id_enc']
        item = row['item_id_enc']
        item_cat = row['item_category_enc']  # static

        # Filter historical data
        hist = train_df[(train_df['shop_id_enc'] == shop) & (train_df['item_id_enc'] == item)]
        hist = hist.sort_values('date')

        # Skip if not enough history
        if len(hist) < window_size:
            continue

        # Take last `window_size` rows
        last_window = hist.iloc[-window_size:].copy()

        # Fill static values to all rows
        last_window['shop_id_enc'] = shop
        last_window['item_id_enc'] = item
        last_window['item_category_enc'] = item_cat

        # Extract features
        seq = last_window[feature_cols].values

        sequences.append(seq)
        valid_indices.append(idx)

    return np.array(sequences), valid_indices



feature_cols = ['amount', 'sales_log', 'price_log',
                'shop_id_enc', 'item_id_enc', 'item_category_enc']
window_size = 12

X_test_sequences, valid_test_indices = get_test_sequences(
    df_test_filtered, df_monthly_lstm_enc2, feature_cols, window_size)

print("Shape of test sequences:", X_test_sequences.shape)
print(f"Number of valid test pairs: {len(valid_test_indices)} / {len(df_test_filtered)}")



Shape of test sequences: (5418, 12, 6)
Number of valid test pairs: 5418 / 24411


In [84]:
from tensorflow.keras.models import load_model
import tensorflow.keras.losses

model_path = '/Users/seydaaybar/Desktop/ntt_data/models/lstm_model.h5'

model = load_model(model_path, custom_objects={'mse': tensorflow.keras.losses.MeanSquaredError()})



In [85]:
# X_test_sequences should be a numpy array of shape (num_samples, window_size, num_features)
predictions = model.predict(X_test_sequences)

# predictions shape: (num_samples, 1)
predictions = predictions.flatten()  # flatten to 1D array


170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [86]:
predictions

array([0.3672012, 0.3672012, 0.3672012, ..., 0.3672012, 0.3672012,
       0.3672012], dtype=float32)

In [87]:
numeric_features = ['amount', 'price_log', 'sales_log']


In [88]:
import joblib

scaler = joblib.load("scaler.pkl")  # Adjust the path if needed


In [89]:
sales_log_idx = numeric_features.index('sales_log')  # Should be 2

dummy = np.zeros((len(predictions), len(numeric_features)))
dummy[:, sales_log_idx] = predictions  # Only fill in sales_log predictions

inv_scaled = scaler.inverse_transform(dummy)
sales_log_inv = inv_scaled[:, sales_log_idx]

sales_pred_actual = np.expm1(sales_log_inv)
sales_pred_actual

array([3209.3729874, 3209.3729874, 3209.3729874, ..., 3209.3729874,
       3209.3729874, 3209.3729874])

In [90]:
print(X_test_sequences.shape)
print(np.std(X_test_sequences, axis=1))  # Should vary across samples


(5418, 12, 6)
[[1.03433612e-01 4.79002444e-01 1.43586207e-04 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [4.53586392e-02 4.96816517e-01 4.56357864e-01 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.58105929e-01 5.65398170e-01 1.35153296e-02 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [3.16211858e-01 5.99879712e-01 1.51886824e-01 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [4.33525146e-01 5.86022502e-01 1.44126328e-01 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.01558207e-01 7.80448805e-01 1.70332947e-01 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


In [91]:
i = 0  # or any index
print(X_test_sequences[i])


[[-1.58608123e-01  7.86172708e+00  7.86172708e+00  2.00000000e+00
   1.42700000e+03  2.10000000e+01]
 [-1.58608123e-01  7.86134180e+00  7.86134180e+00  2.00000000e+00
   1.42700000e+03  2.10000000e+01]
 [-1.58608123e-01  7.86172708e+00  7.86172708e+00  2.00000000e+00
   1.42700000e+03  2.10000000e+01]
 [-3.68981225e-02  8.55468164e+00  7.86172708e+00  2.00000000e+00
   1.42700000e+03  2.10000000e+01]
 [ 8.48118783e-02  8.95969715e+00  7.86134180e+00  2.00000000e+00
   1.42700000e+03  2.10000000e+01]
 [-1.58608123e-01  7.86172708e+00  7.86172708e+00  2.00000000e+00
   1.42700000e+03  2.10000000e+01]
 [ 8.48118783e-02  8.96008253e+00  7.86172708e+00  2.00000000e+00
   1.42700000e+03  2.10000000e+01]
 [ 8.48118783e-02  8.96008253e+00  7.86172708e+00  2.00000000e+00
   1.42700000e+03  2.10000000e+01]
 [-1.58608123e-01  7.86172708e+00  7.86172708e+00  2.00000000e+00
   1.42700000e+03  2.10000000e+01]
 [-1.58608123e-01  7.86172708e+00  7.86172708e+00  2.00000000e+00
   1.42700000e+03  2.1000